In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,shuffle=True)

testloader = torch.utils.data.DataLoader(testset, batch_size=128,shuffle=False)



Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')
input_size=3072
hidden_size1=200
hidden_size2=240
num_classes=10

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)  
        self.relu = nn.ReLU()
        self.fc3=nn.Linear(hidden_size2,num_classes)
    
    def forward(self, x):
        x = x.view(-1, 3072)
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

model = NeuralNet(input_size, hidden_size1,hidden_size2, num_classes)


In [ ]:
num_epochs=25
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.005,weight_decay=0.001)  

total_step = len(trainloader)
for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(trainloader):
           
        
          # Forward pass
          outputs = model(images)
          loss = criterion(outputs, labels)
            
          # Backprpagation and optimization
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
            
          if (i+1) % 128 == 0:

            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/25], Step [128/391], Loss: 1.8678
Epoch [1/25], Step [256/391], Loss: 1.7067
Epoch [1/25], Step [384/391], Loss: 1.7230
Epoch [2/25], Step [128/391], Loss: 1.8282
Epoch [2/25], Step [256/391], Loss: 1.7706
Epoch [2/25], Step [384/391], Loss: 1.6521
Epoch [3/25], Step [128/391], Loss: 1.5646
Epoch [3/25], Step [256/391], Loss: 1.7919
Epoch [3/25], Step [384/391], Loss: 1.7823
Epoch [4/25], Step [128/391], Loss: 1.7802
Epoch [4/25], Step [256/391], Loss: 1.6930
Epoch [4/25], Step [384/391], Loss: 1.6599
Epoch [5/25], Step [128/391], Loss: 1.5779
Epoch [5/25], Step [256/391], Loss: 1.5305
Epoch [5/25], Step [384/391], Loss: 1.6694
Epoch [6/25], Step [128/391], Loss: 1.6063
Epoch [6/25], Step [256/391], Loss: 1.5477
Epoch [6/25], Step [384/391], Loss: 1.4732
Epoch [7/25], Step [128/391], Loss: 1.7112
Epoch [7/25], Step [256/391], Loss: 1.5785
Epoch [7/25], Step [384/391], Loss: 1.6958
Epoch [8/25], Step [128/391], Loss: 1.8340
Epoch [8/25], Step [256/391], Loss: 1.3286
Epoch [8/25

In [ ]:
print(images.shape)

torch.Size([80, 3, 32, 32])


In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the  test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Accuracy of the network on the  test images: 50.4 %


In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in trainloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the  train images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Accuracy of the network on the  train images: 30.604 %


In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 34 %
Accuracy of   car : 71 %
Accuracy of  bird : 39 %
Accuracy of   cat : 14 %
Accuracy of  deer : 33 %
Accuracy of   dog : 45 %
Accuracy of  frog : 50 %
Accuracy of horse : 68 %
Accuracy of  ship : 71 %
Accuracy of truck : 43 %
